In [1]:
%pip install optuna==3.6.1

  Attempting uninstall: optuna
    Found existing installation: optuna 4.0.0
    Uninstalling optuna-4.0.0:
      Successfully uninstalled optuna-4.0.0

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

/home/baldasseroniluisesteban/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib is building the font cache; this may take a moment.


In [ ]:
base_path = '/home/baldasseroniluisesteban/buckets/b1/'
dataset_path = base_path + 'datasets/'


df = pd.read_csv(dataset_path + 'competencia_02_sin_nulos.csv.gz')
df.head()

In [ ]:
#df.info()

In [ ]:
#df.describe()

In [ ]:
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train_inicio = 202002
mes_train_final = 202102
mes_test = 202104

# agregue sus semillas
semillas = [127,181,13]

In [ ]:
# Selección de columnas numéricas para la transformación
data_selected = df[['ctarjeta_visa', 'Visa_cadelantosefectivo', 'cdescubierto_preacordado',
                    'ctrx_quarter', 'ctarjeta_master', 'cproductos', 'Visa_status',
                    'cpayroll_trx']]

# Instanciación de PolynomialFeatures con grado 3
poly = PolynomialFeatures(degree=3, include_bias=False)

# Aplicar la transformación a los datos seleccionados
data_selected_poly = poly.fit_transform(data_selected)

# Crear DataFrame con nuevas características generadas
data_selected_poly_df = pd.DataFrame(data_selected_poly, 
                                     columns=poly.get_feature_names_out(data_selected.columns))

# Renombrar las nuevas columnas para que no coincidan con las originales
data_selected_poly_df.columns = [f"poly_{col}" for col in data_selected_poly_df.columns]

# Concatenar las nuevas columnas al DataFrame original
df_engine = pd.concat([df, data_selected_poly_df], axis=1)


In [ ]:
df_engine['clase_peso'] = 1.0

df_engine.loc[df_engine['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
df_engine.loc[df_engine['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [ ]:
import numpy as np

df_engine['clase_binaria1'] = 0
df_engine['clase_binaria2'] = 0
df_engine['clase_binaria1'] = np.where(df_engine['clase_ternaria'] == 'BAJA+2', 1, 0)
df_engine['clase_binaria2'] = np.where(df_engine['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
train_data_engine = df_engine[(mes_train_inicio <= df_engine['foto_mes']) & (df_engine['foto_mes'] <= mes_train_final)]
test_data_engine = df_engine[df_engine['foto_mes'] == mes_test]

In [ ]:
X_train_engine = train_data_engine.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_train_binaria1_engine = train_data_engine['clase_binaria1']
y_train_binaria2_engine = train_data_engine['clase_binaria2']
w_train_engine = train_data_engine['clase_peso']

X_test_engine = test_data_engine.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_test_binaria1_engine = test_data_engine['clase_binaria1']
y_test_class_engine = test_data_engine['clase_ternaria']
w_test_engine = test_data_engine['clase_peso']

In [ ]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

In [ ]:
import lightgbm as lgb

train_data1_engine = lgb.Dataset(X_train_engine, label=y_train_binaria1_engine, weight=w_train_engine)
train_data2_engine = lgb.Dataset(X_train_engine, label=y_train_binaria2_engine, weight=w_train_engine)

In [ ]:
def objective_engine(trial):
    num_leaves = trial.suggest_int('num_leaves', 100, 500)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 1000)
    feature_fraction = trial.suggest_float('feature_fraction', 0.3, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.3, 1.0)
    max_depth = trial.suggest_int('max_depth', 3, 100)
    bagging_freq = trial.suggest_int('bagging_freq', 1, 300)

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'max_depth': max_depth,
        'bagging_freq': bagging_freq,
        'seed': semillas[0],
        'verbose': -1,
        'early_stopping' :30,
    }
    train_data = lgb.Dataset(X_train_engine,
                              label=y_train_binaria2_engine, # eligir la clase
                              weight=w_train)

    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=50,
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )





    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5


storage_name = "sqlite:///" + dataset_path + "optimization_lgbm_engine.db"
study_name = "exp_101_lgbm_engine"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

In [ ]:
study.optimize(objective_engine, n_trials=50) # subir subir

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
df['clase_peso'] = 1.0

df.loc[df['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
df.loc[df['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [ ]:
df['clase_binaria1'] = 0
df['clase_binaria2'] = 0
df['clase_binaria1'] = np.where(df['clase_ternaria'] == 'BAJA+2', 1, 0)
df['clase_binaria2'] = np.where(df['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
train_data = df[(mes_train_inicio <= df['foto_mes']) & (df['foto_mes'] <= mes_train_final)]
test_data = df[df['foto_mes'] == mes_test]

In [ ]:
X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_train_binaria1 = train_data['clase_binaria1']
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_test_binaria1 = test_data['clase_binaria1']
y_test_class = test_data['clase_ternaria']
w_test = test_data['clase_peso']

In [ ]:
import lightgbm as lgb

train_data1 = lgb.Dataset(X_train, label=y_train_binaria1, weight=w_train)
train_data2 = lgb.Dataset(X_train, label=y_train_binaria2, weight=w_train)

In [ ]:
def objective(trial):
    num_leaves = trial.suggest_int('num_leaves', 100, 500)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 1000)
    feature_fraction = trial.suggest_float('feature_fraction', 0.3, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.3, 1.0)
    max_depth = trial.suggest_int('max_depth', 3, 100)
    bagging_freq = trial.suggest_int('bagging_freq', 1, 300)

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'max_depth': max_depth,
        'bagging_freq': bagging_freq,
        'seed': semillas[0],
        'verbose': -1,
        'early_stopping' :30,
    }
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria2, # eligir la clase
                              weight=w_train)

    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=50,
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )





    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5


storage_name = "sqlite:///" + dataset_path + "optimization_lgbm_base.db"
study_name = "exp_101_lgbm_base"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

In [ ]:
study.optimize(objective, n_trials=50) # subir subir

In [ ]:
optuna.visualization.plot_optimization_history(study)